# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [1]:
import sparker
import random

In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-03-28 10:40:36,546 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-03-28 10:40:36,546 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-28 10:40:36,881 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-28 10:40:37,344 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [4]:
# Profiles contained in the first dataset
profiles = sparker.JSONWrapper.load_profiles('/Final_Project/sample/Df1.json', real_id_field = "id")

Let's visualize a profile to check if they are correctly loaded

In [5]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'title_left', 'value': '495906 b21 hp x5560 2 80ghz ml350 g6 , null new wholesale price'}, {'key': 'description_left', 'value': 'description intel xeon x5560 ml350 g6 2 80ghz 4 core 8mb 95w full processor option kitpart number s option part 495906 b21'}, {'key': 'brand_left', 'value': 'hp enterprise'}, {'key': 'specTableContent_left', 'value': 'specifications category proliant processor sub category xeon generation g6 part number 495906 b21 products id 459573 product type processor upgrade processor type intel xeon processor core quad core processor qty 1 clock speed 2 8ghz bus speed 1333mhz l2 cache 1mb l3 cache 8mb 64 bit processing yes process technology 45nm processor socket socket b lga 1366 thermal design power 95w'}, {'key': 'keyValuePairs_left', 'value': "{'generation': 'g6', 'process technology': '45nm', 'processor type': 'intel xeon', 'processor core': 'quad core', 'product type': 'processor upgrade', 'bus speed': '1333mhz', 'processor

In [6]:
profiles.take(5)
profiles.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{title_left, 495...|          0|         0|        0|
|[{title_left, 495...|          1|         1|        0|
|[{title_left, asu...|          2|         2|        0|
|[{title_left, asu...|          6|         3|        0|
|[{title_left, asu...|          9|         4|        0|
|[{title_left, asr...|         13|         5|        0|
|[{title_left, asr...|         14|         6|        0|
|[{title_left, 533...|         18|         7|        0|
|[{title_left, nul...|         19|         8|        0|
|[{title_left, amd...|         20|         9|        0|
|[{title_left, amd...|         21|        10|        0|
|[{title_left, amd...|         22|        11|        0|
|[{title_left, amd...|         23|        12|        0|
|[{title_left, amd...|         24|        13|        0|
|[{title_left, ben...|         27|        14|   

Extract the max id (it will be used later)

In [7]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

In [8]:
max_profile_id

39797

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [ ]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('../datasets/dirty/cora/groundtruth.csv', 'id1', 'id2')

In [ ]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [ ]:
# We can explore some pairs
random.sample(new_gt, 2)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [142]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 65277


Let's continue by using the standard token blocking

In [9]:
blocks = sparker.Blocking.create_blocks(profiles)
print("Number of blocks",blocks.count())

Number of blocks 38438


## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [10]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [11]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [12]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

NameError: name 'new_gt' is not defined

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [13]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [14]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [15]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 93972677


### Reciprocal Weighted Node Pruning

In [ ]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

### Weighted Edge Pruning

In [ ]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

### Cardinality Node Pruning

In [ ]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

### Reciprocal Cardinality Node Pruning

In [ ]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

### Cardinality Edge Pruning

In [ ]:
results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [16]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 1, 50),
 (0, 2083, 36),
 (0, 22579, 13),
 (0, 22580, 10),
 (0, 22581, 10),
 (0, 22582, 13),
 (0, 22584, 13),
 (0, 22585, 13),
 (0, 22586, 13),
 (0, 22587, 10)]

In [17]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|       7135|            0|[{0, 1, 50}, {0, ...|
|       6008|            0|[{7216, 17194, 33...|
|       5992|            0|[{7232, 17194, 33...|
|       5976|            0|[{7248, 17194, 33...|
|       1207|            0|[{16416, 16417, 2...|
|       3122|            0|[{16496, 16497, 5...|
|       5315|            0|[{128, 7908, 49},...|
|       3206|            0|[{160, 10262, 3},...|
|       2483|            0|[{8432, 18462, 4}...|
|      10666|            0|[{256, 16392, 5},...|
|       6559|            0|[{8448, 12289, 5}...|
|       7924|            0|[{8464, 34819, 11...|
|       3692|            0|[{288, 16621, 6},...|
|       6883|            0|[{8480, 24583, 4}...|
|       2536|            0|[{8544, 10262, 3}...|
|        978|            0|[{16752, 16754, 5...|
|       7273|            0|[{384, 20759, 32}...|
|       5671|       

In [18]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [19]:
max_by_group.take(10)

[(0, 1, 50),
 (7216, 7219, 50),
 (7232, 7235, 50),
 (7248, 7249, 50),
 (16416, 16421, 26),
 (16496, 16512, 54),
 (128, 7908, 49),
 (160, 8100, 27),
 (8432, 8433, 26),
 (256, 8461, 114)]

In [20]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0      1   2
0          0      1  50
1       7216   7219  50
2       7232   7235  50
3       7248   7249  50
4      16416  16421  26
...      ...    ...  ..
39778  11103  11110   7
39779  33247  33248  14
39780   5759  33237  14
39781  20655  21121   6
39782  31199  31200  11

[39783 rows x 3 columns]

In [21]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id   N_id  weight
0      0      1      50
1   7216   7219      50
2   7232   7235      50
3   7248   7249      50
4  16416  16421      26

In [22]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id   N_id  weight
0          0      1      50
1       7216   7219      50
2       7232   7235      50
3       7248   7249      50
4      16416  16421      26
...      ...    ...     ...
39778  11103  11110       7
39779  33247  33248      14
39780   5759  33237      14
39781  20655  21121       6
39782  31199  31200      11

[39783 rows x 3 columns]

In [23]:

new_df = n[(n['weight']>30)]
new_df

P_id   N_id  weight
0          0      1      50
1       7216   7219      50
2       7232   7235      50
3       7248   7249      50
5      16496  16512      54
...      ...    ...     ...
39674  11279  11285      34
39677   7631   7637      34
39678  15823  15827      35
39737  26895  26897      39
39738   6287  35616      36

[28289 rows x 3 columns]

In [24]:
new_df = new_df.sort_values(by=['P_id'])

In [25]:
new_df.head(20)

P_id  N_id  weight
0         0     1      50
2487      1  7216      50
5147      2     3      68
7642      3  7266      68
17411     7  7312      48
27354    11    13      32
29840    12  7345      50
32325    13  7362      32
35830    14    15      33
38323    15    16      33
1045     16    17      33
3509     17    20      33
7643     19  7386      75
10990    20    22      33
15963    22  7385      33
25956    26    28      44
30932    28  7409      44
4107     33  7445      31
24167    41  7489      34
25055    42    43      80

In [26]:
col = ["P_id","N_id","weight"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+
|P_id|N_id|weight|
+----+----+------+
|   0|   1|    50|
|   1|7216|    50|
|   2|   3|    68|
|   3|7266|    68|
|   7|7312|    48|
|  11|  13|    32|
|  12|7345|    50|
|  13|7362|    32|
|  14|  15|    33|
|  15|  16|    33|
|  16|  17|    33|
|  17|  20|    33|
|  19|7386|    75|
|  20|  22|    33|
|  22|7385|    33|
|  26|  28|    44|
|  28|7409|    44|
|  33|7445|    31|
|  41|7489|    34|
|  42|  43|    80|
+----+----+------+
only showing top 20 rows



In [27]:
nid_df = pd.DataFrame(new_df.iloc[:,1])
nid_df

N_id
0          1
2487    7216
5147       3
7642    7266
17411   7312
...      ...
19589  39753
17248  39767
19746  39768
22231  39769
24716  39770

[28289 rows x 1 columns]

In [28]:
ss = SparkSession(sc)


In [39]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .json('hdfs://localhost:9000/Final_Project/sample/Df1.json')
df1.show(5)

+----------+--------------------+-----+--------------------+--------------------+----------+---------------------+--------------------+
|brand_left|    description_left|   id|    identifiers_left|  keyValuePairs_left|price_left|specTableContent_left|          title_left|
+----------+--------------------+-----+--------------------+--------------------+----------+---------------------+--------------------+
|     casio|artikel nr 4216 p...|62286|[{null, null, nul...|                null|      null|                 null|pro trek gungung ...|
|      null|                null|62289|[{null, null, nul...|{null, null, null...|      null| categorie wearabl...|tomtom runner 2 c...|
|      null|                null|62292|[{null, null, nul...|{null, null, null...|      null| categorie wearabl...|tomtom runner 2 c...|
|    tomtom|24 7 activity tra...|62293|[{null, null, nul...|                null|      null|                 null|tomtom runner 2 c...|
|    tomtom|24 7 activity tra...|62294|[{null, n

In [40]:
#rdd_df = df1.rdd.zipWithIndex()

In [41]:
# rdd_df.take(1)

In [ ]:
#df1 = rdd_df.toDF(sampleRatio=0.01)

In [ ]:
# df1.show()

In [42]:
# df1 = df1.withColumn('brand_left', df1['_1'].getItem("brand_left"))
# df1 = df1.withColumn('description_left', df1['_1'].getItem("description_left"))
# df1 = df1.withColumn('id_left', df1['_1'].getItem("id_left"))
# df1 = df1.withColumn('identifiers_left', df1['_1'].getItem("identifiers_left"))
# df1 = df1.withColumn('keyValuePairs_left', df1['_1'].getItem("keyValuePairs_left"))
# df1 = df1.withColumn('specTableContent_left', df1['_1'].getItem("specTableContent_left"))
# df1 = df1.withColumn('title_left', df1['_1'].getItem("title_left"))
# df1 = df1.withColumn('id', df1['_1'].getItem("id"))


# df1 = df1.withColumnRenamed("_2" , "new_id_1")

In [43]:
# df1 = df1.drop("_1")
# df1.show()

In [44]:
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [45]:
pd_common_df = common_df.toPandas()
pd_common_df

brand_left                                   description_left  \
0               canon  top loading nylon camera case if you love your...   
1             sandisk  product description sandisk extreme microsdhc ...   
2               canon  this is considered the standard lens for use w...   
3      action outdoor  produktinformation action outdoor floating han...   
4               casio                                               None   
...               ...                                                ...   
13671            asus  skylake kabylake ddr4 2133 3866 2x pciex16 2x ...   
13672         tp link  the tl sg1024d 24 port gigabit switch provides...   
13673            None  vengeance low profile heat spreaders have a re...   
13674              hp  hp laserjet pro p1102 laser printer monochrome...   
13675  gossammer gear  der gossamer gear gorilla 40 rucksack ist der ...   

          id                                   identifiers_left  \
0       7747  [(None, None, None, None, None, [9320a003], No...   
1       8440  [(None, None, None, None, None, None, None, [1...   
2       8484  [(None, None, None, None, None, [2514a002], No...   
3       9458  [(None, None, None, None, None, None, None, [1...   
4      11567  [(None, None, None, None, None, None, None, [4...   
...      ...                                                ...   
13671  20962  [(None, None, None, None, None, b0s80m0eay...   
13672  18962  [(None, None, None, None, None, [tlsg1024d], N...   
13673  19439  [(None, None, None, [32101602], None, None, No...   
13674  19570  [(None, None, None, None, None, None, [ce651ab...   
13675  21692  [(None, None, None, None, None, None, None, [1...   

                                      keyValuePairs_left price_left  \
0                                                   None       None   
1                                                   None       None   
2                                                   None       None   
3                                                   None       None   
4      (None, None, None, None, None, None, None, Non...       None   
...                                                  ...        ...   
13671                                               None       None   
13672                                               None       None   
13673                                               None     438 89   
13674                                               None       None   
13675                                               None       None   

                                   specTableContent_left  \
0                                                   None   
1                                                   None   
2                                                   None   
3                                                   None   
4      general grupo objetivo hombres carcasa diametr...   
...                                                  ...   
13671                                               None   
13672                                               None   
13673                                               None   
13674                                               None   
13675                                               None   

                                              title_left   P_id   N_id  weight  
0      canon 200dg digital camera gadget bag black ba...   7747   7749      37  
1      sandisk extreme microsdhc 64gb type 10 buy and...   8440   8441      65  
2      canon ef 50mm f 1 8 ii camera lens fixed disco...   8484  22144      45  
3      action outdoor floating hand grip bobber anfug...   9458   9459      34  
4      g shock reloj para hombre casio ga 800 1aer nu...  11567  11570      98  
...                                                  ...    ...    ...     ...  
13671  asus rog strix z270g gaming intel z270 socket ...  20962  20965     155  
13672  tp link 24 port gigabit desktop rackmount swit...  18962  18967     

In [46]:
pd_common_df = pd_common_df.sort_values(by=['P_id'])
pd_common_df

brand_left                                   description_left  \
7873   hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
8345   hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
8767            None                                               None   
8873          asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
8946          asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
...              ...                                                ...   
12165           None                                               None   
11634        corsair  2666mhz ram speed cas 16 18 18 35 timings 1 20...   
11853           None  32gb 2x16gb corsair ddr4 vengeance lpx black p...   
11386           None                                               None   
11497        corsair   ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warranty   

          id                                   identifiers_left  \
7873       0  [(None, None, None, None, None, None, None, [4...   
8345       1  [(None, None, None, None, None, None, None, [4...   
8767       2  [(None, None, None, None, None, [primex299delu...   
8873      13  [(None, None, None, None, None, [asrh81mdgsr2]...   
8946      14  [(None, None, None, None, None, [asrh81mdgsr2]...   
...      ...                                                ...   
12165  39750  [(None, None, None, None, None, [mz7td250bw], ...   
11634  39751  [(None, None, None, None, None, [cmk32gx4m2a26...   
11853  39766  [(None, None, None, None, None, [cmk32gx4m2a26...   
11386  39767  [(None, None, None, None, None, [cmk32gx4m2a26...   
11497  39768  [(None, None, None, None, None, None, [csrlp32...   

                                      keyValuePairs_left price_left  \
7873   (None, None, None, None, None, None, None, Non...       None   
8345   (None, None, None, None, None, None, None, Non...       None   
8767   (None, None, None, None, None, None, None, Non...       None   
8873                                                None       None   
8946                                                None       None   
...                                                  ...        ...   
12165  (None, None, None, None, None, None, None, Non...       None   
11634                                               None       None   
11853  (None, None, None, None, None, None, None, Non...       None   
11386  (None, None, None, None, None, None, None, Non...       None   
11497  (None, None, None, None, None, None, None, Non...       None   

                                   specTableContent_left  \
7873   specifications category proliant processor sub...   
8345   specifications category proliant processor sub...   
8767   categorie moederborden merk asus product asus ...   
8873                                                None   
8946                                                None   
...                                                  ...   
12165  quick specs buffer 256 mb storage capacity 250...   
11634                                               None   
11853  specifications size 32gb 2x16gb memory model v...   
11386  categorie geheugen intern merk corsair product...   
11497  memory type ddr4 pc4 21300 capacity 32gb 2 x 1...   

                                              title_left   P_id   N_id  weight  
7873   495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...      0      1      50  
8345   495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...      1   7216      50  
8767             asus prime x299 deluxe prijzen tweakers      2      3      68  
8873   asrock h81m dgs r2 0 intel h81 socket 1150 ddr...     13   7362      32  
8946   asrock h81m dgs r2 0 intel h81 socket 1150 ddr...     14     15      33  
...                                                  ...    ...    ...     ...  
12165  samsung 840 series 250gb 2 5in sata3 mdx solid...  39750  39752      59  
11634  corsair vengeance lpx 32gb 2x16gb ddr4 pc4 213...  39751  39753      70  
1

In [47]:
pd_common_df = pd_common_df[~pd_common_df.id.isin(nid_df.N_id)]
pd_common_df

brand_left                                   description_left  \
7873   hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
8767            None                                               None   
8946          asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
7869         fujitsu  description fujitsu 146gb ultra320 3 5 inchscs...   
8301            benq  led backlit 1920x1080 resolution 12 000 000 1 ...   
...              ...                                                ...   
10996  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
12007  hp enterprise  description 2 x 72 8gb ultra320 3 5 inch scsi ...   
11686  hp enterprise  description 10 x 72 8gb ultra320 3 5 inch scsi...   
11443  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
11169           None                                               None   

          id                                   identifiers_left  \
7873       0  [(None, None, None, None, None, None, None, [4...   
8767       2  [(None, None, None, None, None, [primex299delu...   
8946      14  [(None, None, None, None, None, [asrh81mdgsr2]...   
7869      19  [(None, None, None, None, None, [mba3147np], N...   
8301      33  [(None, [4718755065736], None, None, None, Non...   
...      ...                                                ...   
10996  39707  [(None, None, None, None, None, None, None, [3...   
12007  39726  [(None, None, None, None, None, None, None, [3...   
11686  39732  [(None, None, None, None, None, None, None, [3...   
11443  39739  [(None, None, None, None, None, None, None, [3...   
11169  39743  [(None, None, None, None, None, [mz7td250bw], ...   

                                      keyValuePairs_left      price_left  \
7873   (None, None, None, None, None, None, None, Non...            None   
8767   (None, None, None, None, None, None, None, Non...            None   
8946                                                None            None   
7869                                                None  389 94 cad cad   
8301                                                None            None   
...                                                  ...             ...   
10996                                               None            None   
12007                                               None            None   
11686                                               None            None   
11443                                               None            None   
11169  (None, None, None, None, None, None, None, Non...            None   

                                   specTableContent_left  \
7873   specifications category proliant processor sub...   
8767   categorie moederborden merk asus product asus ...   
8946                                                None   
7869                                                None   
8301                                                None   
...                                                  ...   
10996                                               None   
12007                                               None   
11686                                               None   
11443                                               None   
11169  quick specs buffer 256 mb storage capacity 250...   

                                              title_left   P_id   N_id  weight  
7873   495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...      0      1      50  
8767             asus prime x299 deluxe prijzen tweakers      2      3      68  
8946   asrock h81m dgs r2 0 intel h81 socket 1150 ddr...     14     15      33  
7869   null , mba3147np fujitsu 146 gb u320 scsi nhp 15k     19   7386      75  
8301   benq zowie rl2455 e sports 24 1920x1080 tn wid...     33   7445      31  
...                                                  ...    ...    ...     ...  
10996  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  39707  39708      40  
12007  null , 306645 0

In [48]:
pd_common_df = pd_common_df.sort_values(by=['id'])
pd_common_df

brand_left                                   description_left  \
7873   hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
8767            None                                               None   
8946          asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
7869         fujitsu  description fujitsu 146gb ultra320 3 5 inchscs...   
8301            benq  led backlit 1920x1080 resolution 12 000 000 1 ...   
...              ...                                                ...   
10996  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
12007  hp enterprise  description 2 x 72 8gb ultra320 3 5 inch scsi ...   
11686  hp enterprise  description 10 x 72 8gb ultra320 3 5 inch scsi...   
11443  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
11169           None                                               None   

          id                                   identifiers_left  \
7873       0  [(None, None, None, None, None, None, None, [4...   
8767       2  [(None, None, None, None, None, [primex299delu...   
8946      14  [(None, None, None, None, None, [asrh81mdgsr2]...   
7869      19  [(None, None, None, None, None, [mba3147np], N...   
8301      33  [(None, [4718755065736], None, None, None, Non...   
...      ...                                                ...   
10996  39707  [(None, None, None, None, None, None, None, [3...   
12007  39726  [(None, None, None, None, None, None, None, [3...   
11686  39732  [(None, None, None, None, None, None, None, [3...   
11443  39739  [(None, None, None, None, None, None, None, [3...   
11169  39743  [(None, None, None, None, None, [mz7td250bw], ...   

                                      keyValuePairs_left      price_left  \
7873   (None, None, None, None, None, None, None, Non...            None   
8767   (None, None, None, None, None, None, None, Non...            None   
8946                                                None            None   
7869                                                None  389 94 cad cad   
8301                                                None            None   
...                                                  ...             ...   
10996                                               None            None   
12007                                               None            None   
11686                                               None            None   
11443                                               None            None   
11169  (None, None, None, None, None, None, None, Non...            None   

                                   specTableContent_left  \
7873   specifications category proliant processor sub...   
8767   categorie moederborden merk asus product asus ...   
8946                                                None   
7869                                                None   
8301                                                None   
...                                                  ...   
10996                                               None   
12007                                               None   
11686                                               None   
11443                                               None   
11169  quick specs buffer 256 mb storage capacity 250...   

                                              title_left   P_id   N_id  weight  
7873   495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...      0      1      50  
8767             asus prime x299 deluxe prijzen tweakers      2      3      68  
8946   asrock h81m dgs r2 0 intel h81 socket 1150 ddr...     14     15      33  
7869   null , mba3147np fujitsu 146 gb u320 scsi nhp 15k     19   7386      75  
8301   benq zowie rl2455 e sports 24 1920x1080 tn wid...     33   7445      31  
...                                                  ...    ...    ...     ...  
10996  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  39707  39708      40  
12007  null , 306645 0

In [49]:
pd_common_df.reset_index(inplace = True)
pd_common_df = pd_common_df.drop(["index"], axis=1)

In [51]:
pd_common_df = pd_common_df.drop(["identifiers_left","keyValuePairs_left","P_id","N_id","weight"], axis =1)
pd_common_df

brand_left                                   description_left     id  \
0     hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...      0   
1              None                                               None      2   
2            asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...     14   
3           fujitsu  description fujitsu 146gb ultra320 3 5 inchscs...     19   
4              benq  led backlit 1920x1080 resolution 12 000 000 1 ...     33   
...             ...                                                ...    ...   
3753  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...  39707   
3754  hp enterprise  description 2 x 72 8gb ultra320 3 5 inch scsi ...  39726   
3755  hp enterprise  description 10 x 72 8gb ultra320 3 5 inch scsi...  39732   
3756  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...  39739   
3757           None                                               None  39743   

          price_left                              specTableContent_left  \
0               None  specifications category proliant processor sub...   
1               None  categorie moederborden merk asus product asus ...   
2               None                                               None   
3     389 94 cad cad                                               None   
4               None                                               None   
...              ...                                                ...   
3753            None                                               None   
3754            None                                               None   
3755            None                                               None   
3756            None                                               None   
3757            None  quick specs buffer 256 mb storage capacity 250...   

                                             title_left  
0     495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...  
1               asus prime x299 deluxe prijzen tweakers  
2     asrock h81m dgs r2 0 intel h81 socket 1150 ddr...  
3     null , mba3147np fujitsu 146 gb u320 scsi nhp 15k  
4     benq zowie rl2455 e sports 24 1920x1080 tn wid...  
...                                                 ...  
3753  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  
3754  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  
3755  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  
3756  null , 306645 003 hp 72 8 gb u320 scsi 15k new...  
3757  samsung 840 series 250gb 2 5in sata3 mdx solid...  

[3758 rows x 6 columns]

In [52]:
pd_common_df["new_id"] = [x for x in range(0,len(pd_common_df))]
pd_common_df = pd_common_df.drop(["id"], axis=1)

In [53]:
pd_common_df

brand_left                                   description_left  \
0     hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
1              None                                               None   
2            asrock  intel h81 chipset x2 ddr3 x1 pci e x16 x1 pci ...   
3           fujitsu  description fujitsu 146gb ultra320 3 5 inchscs...   
4              benq  led backlit 1920x1080 resolution 12 000 000 1 ...   
...             ...                                                ...   
3753  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
3754  hp enterprise  description 2 x 72 8gb ultra320 3 5 inch scsi ...   
3755  hp enterprise  description 10 x 72 8gb ultra320 3 5 inch scsi...   
3756  hp enterprise  description 5 x 72 8gb ultra320 3 5 inch scsi ...   
3757           None                                               None   

          price_left                              specTableContent_left  \
0               None  specifications category proliant processor sub...   
1               None  categorie moederborden merk asus product asus ...   
2               None                                               None   
3     389 94 cad cad                                               None   
4               None                                               None   
...              ...                                                ...   
3753            None                                               None   
3754            None                                               None   
3755            None                                               None   
3756            None                                               None   
3757            None  quick specs buffer 256 mb storage capacity 250...   

                                             title_left  new_id  
0     495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...       0  
1               asus prime x299 deluxe prijzen tweakers       1  
2     asrock h81m dgs r2 0 intel h81 socket 1150 ddr...       2  
3     null , mba3147np fujitsu 146 gb u320 scsi nhp 15k       3  
4     benq zowie rl2455 e sports 24 1920x1080 tn wid...       4  
...                                                 ...     ...  
3753  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3753  
3754  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3754  
3755  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3755  
3756  null , 306645 003 hp 72 8 gb u320 scsi 15k new...    3756  
3757  samsung 840 series 250gb 2 5in sata3 mdx solid...    3757  

[3758 rows x 6 columns]

In [54]:
sp_df1 = spark.createDataFrame(pd_common_df)
sp_df1

DataFrame[brand_left: string, description_left: string, price_left: string, specTableContent_left: string, title_left: string, new_id: bigint]

In [55]:
pd_common_df.to_json('/home/aditi/BigData/AAI_Project/dataset/df1_F1.json')

In [56]:
baseDir = 'hdfs://localhost:9000/Final_Project/sample/DF_op/Df1_F1.json'
sp_df1.write.option("header", True).mode('overwrite').json(baseDir)